# Операции с данными в Pandas

Одной из важнейших частей `NumPy` является возможность проводить быстрые поэлементные операции, как базовые арифметические операции (сложение, вычитание, умножение и т.д.) так и более сложные операции (тригонометрические функции, экспоненциальные и логарифмические функции и т.д.). `Pandas` наследует большую часть такой функциональности от `NumPy` и `ufuncs`, которые были представлены [ранее](https://jakevdp.github.io/PythonDataScienceHandbook/02.03-computation-on-arrays-ufuncs.html), являются ключевой частью этого.

`Pandas` включает пару полезных особенностей, однако: для унарных операций, таких как отрицание и тригонометрические функции, эти `ufuncs` сохраняют индексы и метки колонок в выходных данных, а для бинарных операций, таких как сложение и умножение, `Pandas` автоматически выравнивает индексы при передаче объектов в `ufunc`. Это означает, что сохранение контекста данных и комбинирование данных из различных источников - обе потенциально подверженные ошибкам задачи с сырыми массивами `NumPy` - становятся по существу защищёнными от ошибок в `Pandas`. Мы дополнительно увидим, что существуют чётко определённые операции между одномерными объектами `Series` и двумерными объектами `DataFrame`.

## Ufuncs: сохранение индексов

In [2]:
import pandas as pd
import numpy as np

Ввиду того, что `Pandas` разработан для работы с `NumPy`, любая `ufunc`-функция из `NumPy` будет работать с объектами `Series` или `DataFrame` в `Pandas`.

In [3]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int64

In [4]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)), columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


Если мы применим `ufunc` из `NumPy` на любом из этих объектов, результатом будет другой объект `Pandas` с **сохранёнными индексами**:

In [6]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [7]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


Все функции, которые мы обсуждали [тут](https://jakevdp.github.io/PythonDataScienceHandbook/02.03-computation-on-arrays-ufuncs.html), могут быть примеными аналогичным образом.

## Ufuncs: выравнивание индексов

Для бинарных операций на двух объектах `Series` или `DataFrame`, `Pandas` выровняет индексы во время выполнения операции. Это очень удобно при работе с неполными данными, что мы увидим далее в примерах.

### Выравнивание индексов в `Series`

Допустим мы работаем с двумя наборами данных и нашли данные только о площади только трёх самых больших штатов и о населении только трёх самых населённх штатов.

In [12]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662, 'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193, 'New York': 19651127}, name='population')

In [13]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

Результирующий массив содержит объединение индексов из двух входных массивов, что может быть определено стандартной операцией Python над множествами этих двух индексов:

In [19]:
set(area.index) | set(population.index)

{'Alaska', 'California', 'New York', 'Texas'}

In [20]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

Если для отсутствующих данных `NaN` (not a number) не является желаемым, то можно заменить операцию соответствующим методом. Например, вызов `A.add(B)` является эквивалентом `A + B`, но позволяет явно задать значение для отсутствующих в массиве элементов:

In [21]:
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### Выравнивание индекса в `DataFrame`

Аналогичным образом выравнивание индексов работает для `DataFrame`.

In [24]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)), columns=list('AB'))
A

,A,B
0,0,11
1,11,16


In [25]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)), columns=list('BAC'))
B

,B,A,C
0,9,2,6
1,3,8,2
2,4,2,6


In [26]:
A + B

,A,B,C
0,2.0,20.0,NaN
1,19.0,19.0,NaN
2,NaN,NaN,NaN


Обратите внимание, что индексы корректно выровнены вне зависимости от их порядка в двух исходных объектах и в результирующем объекте индексы отсортированы.

In [61]:
# stack(), в данном случае, превращает DataFrame в Series, содержащий объекты Series
# mean() вычисляет среднее арифметическое для полученного после stack() объекта Series
fill = A.stack().mean()
A.add(B, fill_value=fill)

,A,B,C
0,2.0,20.0,15.5
1,19.0,19.0,11.5
2,11.5,13.5,15.5


В следующей таблице перечислены операторы Python и их методы-эквиваленты для объектов Pandas:

| Python operator | Pandas method(s)                |
|-----------------|---------------------------------|
| `+`             | `add`                           |
| `-`             | `sub`, `subtract`               |
| `*`             | `mul`, `multiply`               |
| `/`             | `truediv`, `div`, `divide`      |
| `//`            | `floordiv`                      |
| `%`             | `mod`                           |
| `**`            | `pow`                           |


## Ufunc: операции между `Series` и `DataFrame`

Во время выполнения операций между объектами `DataFrame` и `Series`, аналогичным образом поддерживается выравнивание индексов и колонок. Операции между объектами `DataFrame` и `Series` аналогичны операциям между двумерными и одномерными массивами `NumPy`. Представьте одну частую операцию, когда мы находим разницу между двумерным массивом и одним из его рядов:

In [78]:
A = rng.randint(100, size=(3, 4))
A

array([[83, 91, 59, 70],
       [43,  7, 46, 34],
       [77, 80, 35, 49]])

In [84]:
# Из каждого ряда двумерного массива A поэлементно вычитаются значения первого ряда (`A[0]`) самого этого массива
# Таким образом, все элементы первого ряда будут равны 0.
A - A[0]

array([[  0,   0,   0,   0],
       [-40, -84, -13, -36],
       [ -6, -11, -24, -21]])

In [93]:
df = pd.DataFrame(A, columns=list('QRST'))
df

,Q,R,S,T
0,83,91,59,70
1,43,7,46,34
2,77,80,35,49


In [95]:
# Вычитание одного ряда из двумерного массива работает аналогично NumPy
# Из каждого ряда массива df (DataFrame) поэлементно вычитаются значения первого ряда (`df.iloc[0]`) самого этого массива
# Таким образом, все элементы первого ряда будут равны 0.
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,-40,-84,-13,-36
2,-6,-11,-24,-21


Если вы хотите выполнять операции не по строкам, а по колонкам, то вы можете воспользоваться соответствующими методами, а не операциями и, с помощью аргумента `axis`, указать вдоль какой оси надо выполнить операцию:

In [99]:
# Из каждой колонки df поэлементно вычесть значения колонки `R` (df['R']), колонка `R` будет содержать нули.
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,-8,0,-32,-21
1,36,0,39,27
2,-3,0,-45,-31


Обратите внимание, что операции между `DataFrame` и `Series`, как и операции рассмотренные ранее, автоматически выравнивают индексы между двумя элементами:

In [105]:
# df.iloc[0] - возвращает первый ряд
# df.iloc[0, ::2] - возвращает каждый второй элемент в первом ряду, начиная с первого (нулевого) элемента
# т.е. следующая операция вернёт половину первого ряда, точнее его чётный значения
halfrow = df.iloc[0, ::2]
halfrow

Q    83
S    59
Name: 0, dtype: int64

In [106]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-40.0,NaN,-13.0,NaN
2,-6.0,NaN,-24.0,NaN


**Итоговый результат всегда будет содержать все индексы и колонки.**